# Spatial Gene Coexpression

---
First, we need to import our spatial data that's been pre-processed:

In [ ]:
source("scripts//spatial_pre_process.R")

We also need to run binSpect to identify genes with a spatial coherent expression profile:

In [ ]:
# identify genes with a spatial coherent expression profile
km_spatialgenes = binSpect(my_giotto_object, bin_method = 'kmeans')

### 1. Spatial correlation
Calculate [spatial correlation scores](https://rubd.github.io/Giotto_site/reference/detectSpatialCorGenes.html):

In [ ]:
ext_spatial_genes = km_spatialgenes[1:500]$genes
spat_cor_netw_DT = detectSpatialCorGenes(my_giotto_object,
                                         method = 'network', 
                                         spatial_network_name = 'Delaunay_network',
                                         subset_genes = ext_spatial_genes)

### 2. Cluster correlation scores:
[Clustering](https://rubd.github.io/Giotto_site/reference/clusterSpatialCorGenes.html):

In [ ]:
spat_cor_netw_DT = clusterSpatialCorGenes(spat_cor_netw_DT, 
                                          name = 'spat_netw_clus', k = 8)
heatmSpatialCorGenes(my_giotto_object, spatCorObject = spat_cor_netw_DT, 
                     use_clus_name = 'spat_netw_clus')

### 3. Network ranks
[Rank](https://rubd.github.io/Giotto_site/reference/rankSpatialCorGroups.html) spatial correlation clusters based on how similar they are:

In [ ]:
netw_ranks = rankSpatialCorGroups(my_giotto_object, 
                                  spatCorObject = spat_cor_netw_DT, 
                                  use_clus_name = 'spat_netw_clus')

### 4. Cluster information
See info about [clusters](https://rubd.github.io/Giotto_site/reference/showSpatialCorGenes.html):

In [ ]:
top_netw_spat_cluster = showSpatialCorGenes(spat_cor_netw_DT, 
                                            use_clus_name = 'spat_netw_clus',
                                            selected_clusters = 6, 
                                            show_top_genes = 1)

cluster_genes_DT = showSpatialCorGenes(spat_cor_netw_DT, 
                                       use_clus_name = 'spat_netw_clus',
                                       show_top_genes = 1)

cluster_genes = cluster_genes_DT$clus; names(cluster_genes) = cluster_genes_DT$gene_ID

### 5. Visualize
First we want to [create metagenes](https://rubd.github.io/Giotto_site/reference/createMetagenes.html) and then visualize:

In [ ]:
my_giotto_object = createMetagenes(my_giotto_object, gene_clusters = cluster_genes, name = 'cluster_metagene')
spatCellPlot(my_giotto_object,
             spat_enr_names = 'cluster_metagene',
             cell_annotation_values = netw_ranks$clusters,
             point_size = 1.5, cow_n_col = 3)[0]